In [15]:
print("----\n")

[MyPython] Info:The process :h:\Jupyter\Myjupyter-kernel\kernel\C\tmprmznogke.py


[MyPython] Info:python3 h:\Jupyter\Myjupyter-kernel\kernel\C\tmprmznogke.py 


----



The process end:7628


In [16]:
##%overwritefile
##%file:src/compile_with_gcc.py
##%noruncode
    def compile_with_gcc(self, source_filename, binary_filename, cflags=None, ldflags=None,env=None,magics=None):
        # cflags = ['-std=c89', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-Wdeclaration-after-statement', '-Wvla', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=iso9899:199409', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c11', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        outfile=binary_filename
        orig_cflags=cflags
        orig_ldflags=ldflags
        if self.linkMaths:
            cflags = cflags + ['-lm']
        if self.wError:
            cflags = cflags + ['-Werror']
        if self.wAll:
            cflags = cflags + ['-Wall']
        if self.readOnlyFileSystem:
            cflags = ['-DREAD_ONLY_FILE_SYSTEM'] + cflags
        if self.bufferedOutput:
            cflags = ['-DBUFFERED_OUTPUT'] + cflags

        for s in cflags:
                if s.startswith('-o'):
                    if(len(s)>2):
                        outfile=s[2:]
                    else:
                        outfile=cflags[cflags.index('-o')+1]
                        if outfile.startswith('-'):
                            outfile=binary_filename
                binary_filename=outfile
        args=[]
        if magics!=None and len(self.addkey2dict(magics,'ccompiler'))>0:
            args = magics['ccompiler'] + orig_cflags +[source_filename] + orig_ldflags
        else:
            args = ['gcc', source_filename] + ['-o', binary_filename]+ cflags  + ldflags
        self._log(''.join((' '+ str(s) for s in args))+"\n")
        return self.create_jupyter_subprocess(args,env=env),binary_filename,args


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\C\src/compile_with_gcc.py created successfully


In [17]:
##%overwritefile
##%file:src/c_exec_gcc_.py
##%noruncode
    def _exec_gcc_(self,source_filename,magics):
        self._logln('Generating executable file')
        with self.new_temp_file(suffix='.out') as binary_file:
            p,outfile,gcccmd = self.compile_with_gcc(
                source_filename, 
                binary_file.name,
                self.addkey2dict(magics,'cflags'),
                self.addkey2dict(magics,'ldflags'),
                self.addkey2dict(magics,'env'),
                magics=magics)
            while p.poll() is None:
                p.write_contents()
            p.write_contents()
            binary_file.name=os.path.join(os.path.abspath(''),outfile)
            if p.returncode != 0:  # Compilation failed
                self._log(''.join((str(s) for s in gcccmd))+"\n",3)
                self._log("C compiler exited with code {}, the executable will not be executed".format(p.returncode),3)

                # delete source files before exit
                os.remove(source_filename)
                os.remove(binary_file.name)
        return p.returncode,binary_file.name


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\C\src/c_exec_gcc_.py created successfully
